In [7]:
# load data
import pandas as pd
import time
start_time = time.time()   
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Start Time =", current_time)

windows_filepath = 'D:/labelDataOffset.csv'
df = pd.read_csv(windows_filepath,header=None)

print("--- %s seconds ---" % round((time.time() - start_time),2))
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("End Time =", current_time)

Start Time = 22:11:42
--- 26.09 seconds ---
End Time = 22:12:09


In [4]:
# X and y split
X = df.iloc[:,:512]
y = df[512]

EDA

In [5]:
import seaborn as sns
print(y.value_counts())

7     85221
8     80236
4     60263
1     46822
6     36718
2     36631
5     33911
9     33591
3     26019
10    25368
Name: 512, dtype: int64


Machine Learning Modelling with 10 fold cross validation
Store prediction probability score for each label

In [19]:
# XGBoost

from sklearn.model_selection import StratifiedKFold, KFold
kf = KFold(n_splits=10, shuffle=True)

i=0

for train_index, test_index in kf.split(X, y):
    i+=1
    import time
    start_time = time.time()   
    from datetime import datetime
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Start Time =", current_time)
    
    # assign training and testing set
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # fit model with a transformation pipeline
    print('fitting XGBoost Model...') 
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import StandardScaler
    import xgboost as xgb
    pipe = Pipeline([('scaler', StandardScaler()), ('xgb', xgb.XGBClassifier())])
    pipe.fit(X_train, y_train)
    
    
    # accuracy metrics
    print('calculating accuracy on test...')
    from sklearn.metrics import accuracy_score
    print(f'model accuracy is {accuracy_score(y_test, pipe.predict(X_test))}') 

    # calculate and save predict probability to csv file
    print('calculating predict probability and saving with index to a csv file')
    y_pred_prob = pipe.predict_proba(X_test)
    
    y_pred_prob_df = pd.DataFrame(y_pred_prob)
    y_pred_prob_df['index'] = test_index
    y_pred_prob_df.to_csv(f'D:/xgb_pred_prob_test_index_{i}.csv')
    
    # save the model to disk
    print('saving the model to disk...')
    import joblib
    joblib.dump(pipe, f'D:/xgb_labelDataOffset_{i}.sav')
    
    print("--- %s seconds ---" % (time.time() - start_time))
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")


Start Time = 23:49:01
fitting XGBoost Model...
calculating accuracy on test...
model accuracy is 0.5099832178665175
calculating predict probability and saving with index to a csv file
saving the model to disk...
--- 2764.666208744049 seconds ---
Start Time = 00:35:06
fitting XGBoost Model...
calculating accuracy on test...
model accuracy is 0.5112741512113258
calculating predict probability and saving with index to a csv file
saving the model to disk...
--- 2803.3638513088226 seconds ---
Start Time = 01:21:49
fitting XGBoost Model...
calculating accuracy on test...
model accuracy is 0.5113171823228194
calculating predict probability and saving with index to a csv file
saving the model to disk...
--- 2772.617698431015 seconds ---
Start Time = 02:08:02
fitting XGBoost Model...
calculating accuracy on test...
model accuracy is 0.5140496579026637
calculating predict probability and saving with index to a csv file
saving the model to disk...
--- 2772.3251435756683 seconds ---
Start Time = 0

In [20]:
# CatBoost

from sklearn.model_selection import StratifiedKFold, KFold
kf = KFold(n_splits=10, shuffle=True)

i=0

for train_index, test_index in kf.split(X, y):
    i+=1
    import time
    start_time = time.time()   
    from datetime import datetime
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Start Time =", current_time)
    
    # assign training and testing set
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # fit model with a transformation pipeline
    print('fitting CatBoost Model...') 
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import StandardScaler
    from catboost import CatBoostClassifier
    pipe = Pipeline([('scaler', StandardScaler()), ('catboost', CatBoostClassifier())])
    pipe.fit(X_train, y_train)
    
    # accuracy metrics
    print('calculating accuracy on test...')
    from sklearn.metrics import accuracy_score
    print(f'model accuracy is {accuracy_score(y_test, pipe.predict(X_test))}') 
    
    
    # calculate and save predict probability to csv file
    print('calculating predict probability and saving with index to a csv file')
    y_pred_prob = pipe.predict_proba(X_test)
    y_pred_prob_df = pd.DataFrame(y_pred_prob)
    y_pred_prob_df['index'] = test_index
    y_pred_prob_df.to_csv(f'D:/catboost_pred_prob_test_index_{i}.csv')
    
    # save the model to disk
    print('saving the model to disk...')
    import joblib
    joblib.dump(pipe, f'D:/catboost_labelDataOffset_{i}.sav')
    
    print("--- %s seconds ---" % (time.time() - start_time))
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")


Start Time = 07:26:27
fitting CatBoost Model...
Learning rate set to 0.107651
0:	learn: 2.2151360	total: 3.02s	remaining: 50m 15s
1:	learn: 2.1560164	total: 6.08s	remaining: 50m 36s
2:	learn: 2.1025930	total: 9.08s	remaining: 50m 17s
3:	learn: 2.0608717	total: 12s	remaining: 49m 53s
4:	learn: 2.0280833	total: 15.1s	remaining: 50m 1s
5:	learn: 1.9699717	total: 18.2s	remaining: 50m 7s
6:	learn: 1.9400501	total: 21.1s	remaining: 49m 54s
7:	learn: 1.9148453	total: 24.1s	remaining: 49m 44s
8:	learn: 1.8926536	total: 27.1s	remaining: 49m 41s
9:	learn: 1.8533377	total: 30.2s	remaining: 49m 45s
10:	learn: 1.8397688	total: 33.2s	remaining: 49m 42s
11:	learn: 1.8073656	total: 36.2s	remaining: 49m 37s
12:	learn: 1.7840612	total: 39.1s	remaining: 49m 31s
13:	learn: 1.7544837	total: 42.1s	remaining: 49m 28s
14:	learn: 1.7277824	total: 45.2s	remaining: 49m 27s
15:	learn: 1.7167286	total: 48.2s	remaining: 49m 26s
16:	learn: 1.6953799	total: 51.2s	remaining: 49m 23s
17:	learn: 1.6897366	total: 54.2s	r

151:	learn: 1.4034036	total: 7m 35s	remaining: 42m 22s
152:	learn: 1.4028132	total: 7m 38s	remaining: 42m 19s
153:	learn: 1.4021505	total: 7m 41s	remaining: 42m 16s
154:	learn: 1.4014114	total: 7m 44s	remaining: 42m 13s
155:	learn: 1.4001198	total: 7m 47s	remaining: 42m 10s
156:	learn: 1.3996161	total: 7m 50s	remaining: 42m 7s
157:	learn: 1.3992543	total: 7m 53s	remaining: 42m 4s
158:	learn: 1.3987520	total: 7m 56s	remaining: 42m
159:	learn: 1.3979555	total: 7m 59s	remaining: 41m 58s
160:	learn: 1.3974494	total: 8m 2s	remaining: 41m 54s
161:	learn: 1.3970127	total: 8m 5s	remaining: 41m 51s
162:	learn: 1.3968306	total: 8m 8s	remaining: 41m 47s
163:	learn: 1.3963832	total: 8m 11s	remaining: 41m 44s
164:	learn: 1.3958608	total: 8m 14s	remaining: 41m 41s
165:	learn: 1.3955448	total: 8m 17s	remaining: 41m 38s
166:	learn: 1.3946239	total: 8m 20s	remaining: 41m 35s
167:	learn: 1.3935650	total: 8m 23s	remaining: 41m 32s
168:	learn: 1.3916933	total: 8m 26s	remaining: 41m 29s
169:	learn: 1.39130

300:	learn: 1.2970376	total: 15m 2s	remaining: 34m 54s
301:	learn: 1.2961306	total: 15m 5s	remaining: 34m 51s
302:	learn: 1.2957999	total: 15m 8s	remaining: 34m 48s
303:	learn: 1.2956849	total: 15m 11s	remaining: 34m 45s
304:	learn: 1.2949945	total: 15m 14s	remaining: 34m 42s
305:	learn: 1.2945279	total: 15m 17s	remaining: 34m 39s
306:	learn: 1.2942701	total: 15m 19s	remaining: 34m 36s
307:	learn: 1.2931708	total: 15m 23s	remaining: 34m 33s
308:	learn: 1.2929591	total: 15m 25s	remaining: 34m 30s
309:	learn: 1.2921969	total: 15m 29s	remaining: 34m 28s
310:	learn: 1.2910276	total: 15m 32s	remaining: 34m 25s
311:	learn: 1.2904077	total: 15m 35s	remaining: 34m 22s
312:	learn: 1.2902186	total: 15m 38s	remaining: 34m 19s
313:	learn: 1.2892600	total: 15m 41s	remaining: 34m 16s
314:	learn: 1.2890435	total: 15m 44s	remaining: 34m 13s
315:	learn: 1.2887497	total: 15m 47s	remaining: 34m 10s
316:	learn: 1.2885806	total: 15m 50s	remaining: 34m 7s
317:	learn: 1.2870046	total: 15m 53s	remaining: 34m 

448:	learn: 1.2322917	total: 22m 26s	remaining: 27m 32s
449:	learn: 1.2317688	total: 22m 29s	remaining: 27m 29s
450:	learn: 1.2316588	total: 22m 32s	remaining: 27m 26s
451:	learn: 1.2313745	total: 22m 35s	remaining: 27m 23s
452:	learn: 1.2309068	total: 22m 38s	remaining: 27m 20s
453:	learn: 1.2307758	total: 22m 41s	remaining: 27m 17s
454:	learn: 1.2303771	total: 22m 44s	remaining: 27m 14s
455:	learn: 1.2299264	total: 22m 47s	remaining: 27m 11s
456:	learn: 1.2296757	total: 22m 50s	remaining: 27m 8s
457:	learn: 1.2292000	total: 22m 53s	remaining: 27m 5s
458:	learn: 1.2288103	total: 22m 56s	remaining: 27m 2s
459:	learn: 1.2282228	total: 22m 59s	remaining: 26m 59s
460:	learn: 1.2277593	total: 23m 2s	remaining: 26m 56s
461:	learn: 1.2275234	total: 23m 5s	remaining: 26m 53s
462:	learn: 1.2272990	total: 23m 8s	remaining: 26m 50s
463:	learn: 1.2266136	total: 23m 11s	remaining: 26m 47s
464:	learn: 1.2255290	total: 23m 14s	remaining: 26m 44s
465:	learn: 1.2252223	total: 23m 17s	remaining: 26m 41

596:	learn: 1.1852688	total: 29m 51s	remaining: 20m 9s
597:	learn: 1.1851845	total: 29m 54s	remaining: 20m 6s
598:	learn: 1.1849044	total: 29m 57s	remaining: 20m 3s
599:	learn: 1.1847814	total: 29m 59s	remaining: 19m 59s
600:	learn: 1.1843935	total: 30m 2s	remaining: 19m 56s
601:	learn: 1.1842439	total: 30m 5s	remaining: 19m 53s
602:	learn: 1.1841340	total: 30m 8s	remaining: 19m 51s
603:	learn: 1.1840065	total: 30m 11s	remaining: 19m 47s
604:	learn: 1.1838998	total: 30m 14s	remaining: 19m 44s
605:	learn: 1.1837710	total: 30m 17s	remaining: 19m 41s
606:	learn: 1.1836614	total: 30m 20s	remaining: 19m 38s
607:	learn: 1.1834944	total: 30m 23s	remaining: 19m 35s
608:	learn: 1.1830982	total: 30m 26s	remaining: 19m 32s
609:	learn: 1.1827694	total: 30m 29s	remaining: 19m 29s
610:	learn: 1.1826779	total: 30m 32s	remaining: 19m 26s
611:	learn: 1.1825036	total: 30m 35s	remaining: 19m 23s
612:	learn: 1.1822839	total: 30m 38s	remaining: 19m 20s
613:	learn: 1.1818740	total: 30m 41s	remaining: 19m 17

...

371:	learn: 1.2613264	total: 18m 25s	remaining: 31m 5s
372:	learn: 1.2605630	total: 18m 28s	remaining: 31m 2s
373:	learn: 1.2602858	total: 18m 31s	remaining: 31m
374:	learn: 1.2596482	total: 18m 34s	remaining: 30m 57s
375:	learn: 1.2594228	total: 18m 37s	remaining: 30m 54s
376:	learn: 1.2592886	total: 18m 40s	remaining: 30m 51s
377:	learn: 1.2585505	total: 18m 43s	remaining: 30m 48s
378:	learn: 1.2580292	total: 18m 46s	remaining: 30m 45s
379:	learn: 1.2575607	total: 18m 49s	remaining: 30m 42s
380:	learn: 1.2574135	total: 18m 52s	remaining: 30m 39s
381:	learn: 1.2572020	total: 18m 54s	remaining: 30m 36s
382:	learn: 1.2565899	total: 18m 57s	remaining: 30m 33s
383:	learn: 1.2563944	total: 19m	remaining: 30m 30s
384:	learn: 1.2562134	total: 19m 3s	remaining: 30m 27s
385:	learn: 1.2556701	total: 19m 7s	remaining: 30m 24s
386:	learn: 1.2550628	total: 19m 10s	remaining: 30m 21s
387:	learn: 1.2545536	total: 19m 12s	remaining: 30m 18s
388:	learn: 1.2543059	total: 19m 15s	remaining: 30m 15s
389:

519:	learn: 1.2069705	total: 25m 46s	remaining: 23m 47s
520:	learn: 1.2066339	total: 25m 49s	remaining: 23m 44s
521:	learn: 1.2061784	total: 25m 52s	remaining: 23m 41s
522:	learn: 1.2057054	total: 25m 55s	remaining: 23m 38s
523:	learn: 1.2053585	total: 25m 58s	remaining: 23m 35s
524:	learn: 1.2051440	total: 26m 1s	remaining: 23m 32s
525:	learn: 1.2046419	total: 26m 4s	remaining: 23m 29s
526:	learn: 1.2045360	total: 26m 7s	remaining: 23m 26s
527:	learn: 1.2043771	total: 26m 10s	remaining: 23m 23s
528:	learn: 1.2043069	total: 26m 12s	remaining: 23m 20s
529:	learn: 1.2037982	total: 26m 16s	remaining: 23m 17s
530:	learn: 1.2029903	total: 26m 19s	remaining: 23m 14s
531:	learn: 1.2024933	total: 26m 22s	remaining: 23m 11s
532:	learn: 1.2019349	total: 26m 25s	remaining: 23m 8s
533:	learn: 1.2017944	total: 26m 27s	remaining: 23m 5s
534:	learn: 1.2014681	total: 26m 31s	remaining: 23m 2s
535:	learn: 1.2013284	total: 26m 34s	remaining: 22m 59s
536:	learn: 1.2010109	total: 26m 37s	remaining: 22m 57

667:	learn: 1.1670082	total: 33m 7s	remaining: 16m 27s
668:	learn: 1.1668703	total: 33m 10s	remaining: 16m 24s
669:	learn: 1.1666445	total: 33m 13s	remaining: 16m 21s
670:	learn: 1.1662922	total: 33m 16s	remaining: 16m 18s
671:	learn: 1.1662034	total: 33m 19s	remaining: 16m 15s
672:	learn: 1.1660742	total: 33m 22s	remaining: 16m 12s
673:	learn: 1.1657290	total: 33m 25s	remaining: 16m 9s
674:	learn: 1.1656160	total: 33m 28s	remaining: 16m 6s
675:	learn: 1.1654513	total: 33m 31s	remaining: 16m 3s
676:	learn: 1.1652923	total: 33m 34s	remaining: 16m
677:	learn: 1.1648323	total: 33m 37s	remaining: 15m 57s
678:	learn: 1.1644378	total: 33m 40s	remaining: 15m 55s
679:	learn: 1.1643604	total: 33m 43s	remaining: 15m 52s
680:	learn: 1.1639112	total: 33m 46s	remaining: 15m 49s
681:	learn: 1.1638420	total: 33m 49s	remaining: 15m 46s
682:	learn: 1.1637285	total: 33m 51s	remaining: 15m 43s
683:	learn: 1.1634747	total: 33m 54s	remaining: 15m 40s
684:	learn: 1.1630830	total: 33m 57s	remaining: 15m 37s


815:	learn: 1.1371131	total: 40m 29s	remaining: 9m 7s
816:	learn: 1.1370218	total: 40m 31s	remaining: 9m 4s
817:	learn: 1.1369114	total: 40m 34s	remaining: 9m 1s
818:	learn: 1.1366947	total: 40m 37s	remaining: 8m 58s
819:	learn: 1.1364244	total: 40m 40s	remaining: 8m 55s
820:	learn: 1.1361417	total: 40m 43s	remaining: 8m 52s
821:	learn: 1.1357842	total: 40m 46s	remaining: 8m 49s
822:	learn: 1.1356710	total: 40m 49s	remaining: 8m 46s
823:	learn: 1.1354495	total: 40m 52s	remaining: 8m 43s
824:	learn: 1.1351276	total: 40m 55s	remaining: 8m 40s
825:	learn: 1.1350360	total: 40m 58s	remaining: 8m 37s
826:	learn: 1.1348374	total: 41m 1s	remaining: 8m 34s
827:	learn: 1.1347340	total: 41m 4s	remaining: 8m 31s
828:	learn: 1.1344983	total: 41m 7s	remaining: 8m 29s
829:	learn: 1.1343727	total: 41m 10s	remaining: 8m 26s
830:	learn: 1.1341335	total: 41m 13s	remaining: 8m 23s
831:	learn: 1.1338193	total: 41m 16s	remaining: 8m 20s
832:	learn: 1.1336628	total: 41m 19s	remaining: 8m 17s
833:	learn: 1.13

966:	learn: 1.1093874	total: 48m	remaining: 1m 38s
967:	learn: 1.1092886	total: 48m 3s	remaining: 1m 35s
968:	learn: 1.1092023	total: 48m 6s	remaining: 1m 32s
969:	learn: 1.1091297	total: 48m 9s	remaining: 1m 29s
970:	learn: 1.1088936	total: 48m 12s	remaining: 1m 26s
971:	learn: 1.1088391	total: 48m 15s	remaining: 1m 23s
972:	learn: 1.1087794	total: 48m 18s	remaining: 1m 20s
973:	learn: 1.1086222	total: 48m 21s	remaining: 1m 17s
974:	learn: 1.1085188	total: 48m 24s	remaining: 1m 14s
975:	learn: 1.1082593	total: 48m 27s	remaining: 1m 11s
976:	learn: 1.1078994	total: 48m 30s	remaining: 1m 8s
977:	learn: 1.1076991	total: 48m 33s	remaining: 1m 5s
978:	learn: 1.1075959	total: 48m 36s	remaining: 1m 2s
979:	learn: 1.1074210	total: 48m 39s	remaining: 59.6s
980:	learn: 1.1073522	total: 48m 42s	remaining: 56.6s
981:	learn: 1.1071634	total: 48m 45s	remaining: 53.6s
982:	learn: 1.1071165	total: 48m 48s	remaining: 50.6s
983:	learn: 1.1069803	total: 48m 51s	remaining: 47.7s
984:	learn: 1.1069227	tot

In [22]:
from sklearn.model_selection import StratifiedKFold, KFold
kf = KFold(n_splits=10, shuffle=True)

i=0

for train_index, test_index in kf.split(X, y):
    i+=1
    import time
    start_time = time.time()   
    from datetime import datetime
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Start Time =", current_time)
    
    X_train, X_test = X_std.iloc[train_index], X_std.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]


    # fit model with a transformation pipeline
    print('fitting Light GBM Model...') 
    import lightgbm as lgb
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import StandardScaler
    pipe = Pipeline([('scaler', StandardScaler()), ('lgb', lgb.LGBMClassifier())])
    pipe.fit(X_train, y_train)
    
    # accuracy metrics
    print('calculating accuracy score...')
    from sklearn.metrics import accuracy_score
    print(f'model accuracy is {accuracy_score(y_test, pipe.predict(X_test))}') 
    
    # calculate and save predict probability to csv file
    print('calculating predict probability and saving with index to a csv file')
    y_pred_prob = pipe.predict_proba(X_test)
    y_pred_prob_df = pd.DataFrame(y_pred_prob)
    y_pred_prob_df['index'] = test_index
    y_pred_prob_df.to_csv(f'D:/lgbm_pred_prob_test_index_{i}.csv')
    
    # save the model to disk
    print('saving the model to disk...')
    import joblib
    joblib.dump(pipe, f'D:/lgbm_labelDataOffset_{i}.sav')
    
    print("--- %s seconds ---" % (time.time() - start_time))
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")

Start Time = 15:56:35
fitting Light GBM Model...
calculating accuracy score...
model accuracy is 0.4602607685356513
calculating predict probability and saving with index to a csv file
saving the model to disk...
--- 151.14159488677979 seconds ---
Start Time = 15:59:06
fitting Light GBM Model...
calculating accuracy score...
model accuracy is 0.46021773742415767
calculating predict probability and saving with index to a csv file
saving the model to disk...
--- 151.33013939857483 seconds ---
Start Time = 16:01:37
fitting Light GBM Model...
calculating accuracy score...
model accuracy is 0.4599165196437024
calculating predict probability and saving with index to a csv file
saving the model to disk...
--- 150.19042110443115 seconds ---
Start Time = 16:04:08
fitting Light GBM Model...
calculating accuracy score...
model accuracy is 0.4617453418821808
calculating predict probability and saving with index to a csv file
saving the model to disk...
--- 150.47288274765015 seconds ---
Start Time 

In [24]:
# Random Forest
model = 'Random_Forest'
from sklearn.model_selection import StratifiedKFold, KFold
kf = KFold(n_splits=10, shuffle=True)

i=0

for train_index, test_index in kf.split(X, y):
    i+=1
    import time
    start_time = time.time()   
    from datetime import datetime
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Start Time =", current_time)
    
    # assign training and testing set
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # fit model with a transformation pipeline
    print(f'fitting {model} Model...') 
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import StandardScaler
    from sklearn.ensemble import RandomForestClassifier
    pipe = Pipeline([('scaler', StandardScaler()), (f'{model}', RandomForestClassifier(n_jobs=-1))])
    pipe.fit(X_train, y_train)
    
    # accuracy metrics
    print('calculating accuracy on test...')
    from sklearn.metrics import accuracy_score
    print(f'model accuracy is {accuracy_score(y_test, pipe.predict(X_test))}') 
    
    
    # calculate and save predict probability to csv file
    print('calculating predict probability and saving with index to a csv file')
    y_pred_prob = pipe.predict_proba(X_test)
    y_pred_prob_df = pd.DataFrame(y_pred_prob)
    y_pred_prob_df['index'] = test_index
    y_pred_prob_df.to_csv(f'D:/{model}_pred_prob_test_index_{i}.csv')
    
    # save the model to disk
    print('saving the model to disk...')
    import joblib
    joblib.dump(pipe, f'D:/{model}_labelDataOffset_{i}.sav')
    
    print("--- %s seconds ---" % (time.time() - start_time))
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")


Start Time = 21:06:02
fitting Random_Forest Model...
calculating accuracy on test...
model accuracy is 0.5673006583760059
calculating predict probability and saving with index to a csv file
saving the model to disk...
--- 173.33036160469055 seconds ---
Start Time = 21:08:55
fitting Random_Forest Model...
calculating accuracy on test...
model accuracy is 0.5706355695167606
calculating predict probability and saving with index to a csv file
saving the model to disk...
--- 171.53793859481812 seconds ---
Start Time = 21:11:46
fitting Random_Forest Model...
calculating accuracy on test...
model accuracy is 0.5692800895047119
calculating predict probability and saving with index to a csv file
saving the model to disk...
--- 173.5499725341797 seconds ---
Start Time = 21:14:40
fitting Random_Forest Model...
calculating accuracy on test...
model accuracy is 0.5647833383536297
calculating predict probability and saving with index to a csv file
saving the model to disk...
--- 172.65745306015015 s

In [25]:
# KNN
model = 'KNN'
from sklearn.model_selection import StratifiedKFold, KFold
kf = KFold(n_splits=10, shuffle=True)

i=0

for train_index, test_index in kf.split(X, y):
    i+=1
    import time
    start_time = time.time()   
    from datetime import datetime
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Start Time =", current_time)
    
    # assign training and testing set
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # fit model with a transformation pipeline
    print(f'fitting {model} Model...') 
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import StandardScaler
    from sklearn.neighbors import KNeighborsClassifier
    pipe = Pipeline([('scaler', StandardScaler()), (f'{model}', KNeighborsClassifier(n_jobs = -1) )])
    pipe.fit(X_train, y_train)
    
    # accuracy metrics
    print('calculating accuracy on test...')
    from sklearn.metrics import accuracy_score
    print(f'model accuracy is {accuracy_score(y_test, pipe.predict(X_test))}') 
    
    
    # calculate and save predict probability to csv file
    print('calculating predict probability and saving with index to a csv file')
    y_pred_prob = pipe.predict_proba(X_test)
    y_pred_prob_df = pd.DataFrame(y_pred_prob)
    y_pred_prob_df['index'] = test_index
    y_pred_prob_df.to_csv(f'D:/{model}_pred_prob_test_index_{i}.csv')
    
    # save the model to disk
    print('saving the model to disk...')
    import joblib
    joblib.dump(pipe, f'D:/{model}_labelDataOffset_{i}.sav')
    
    print("--- %s seconds ---" % (time.time() - start_time))
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")

Start Time = 21:34:49
fitting KNN Model...
calculating accuracy on test...
model accuracy is 0.5100692800895047
calculating predict probability and saving with index to a csv file
saving the model to disk...
--- 165.06094932556152 seconds ---
Start Time = 21:37:34
fitting KNN Model...
calculating accuracy on test...
model accuracy is 0.5070786178406989
calculating predict probability and saving with index to a csv file
saving the model to disk...
--- 163.41057658195496 seconds ---
Start Time = 21:40:17
fitting KNN Model...
calculating accuracy on test...
model accuracy is 0.5129523645595766
calculating predict probability and saving with index to a csv file
saving the model to disk...
--- 163.82254815101624 seconds ---
Start Time = 21:43:01
fitting KNN Model...
calculating accuracy on test...
model accuracy is 0.5081543956280391
calculating predict probability and saving with index to a csv file
saving the model to disk...
--- 163.97091913223267 seconds ---
Start Time = 21:45:45
fittin

In [30]:
# logistic_regression
model = 'logistic_regression'
from sklearn.model_selection import StratifiedKFold, KFold
kf = KFold(n_splits=10, shuffle=True)

i=0

for train_index, test_index in kf.split(X, y):
    i+=1
    import time
    start_time = time.time()   
    from datetime import datetime
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Start Time =", current_time)
    
    # assign training and testing set
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # fit model with a transformation pipeline
    print(f'fitting {model} Model...') 
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import StandardScaler
    from sklearn.linear_model import LogisticRegression
    pipe = Pipeline([('scaler', StandardScaler()), (f'{model}', LogisticRegression(n_jobs = -1) )])
    pipe.fit(X_train, y_train)
    
    # accuracy metrics
    print('calculating accuracy on test...')
    from sklearn.metrics import accuracy_score
    print(f'model accuracy is {accuracy_score(y_test, pipe.predict(X_test))}') 
    
    
    # calculate and save predict probability to csv file
    print('calculating predict probability and saving with index to a csv file')
    y_pred_prob = pipe.predict_proba(X_test)
    y_pred_prob_df = pd.DataFrame(y_pred_prob)
    y_pred_prob_df['index'] = test_index
    y_pred_prob_df.to_csv(f'D:/{model}_pred_prob_test_index_{i}.csv')
    
    # save the model to disk
    print('saving the model to disk...')
    import joblib
    joblib.dump(pipe, f'D:/{model}_labelDataOffset_{i}.sav')
    
    print("--- %s seconds ---" % (time.time() - start_time))
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")

Start Time = 23:51:07
fitting logistic_regression Model...
calculating accuracy on test...
model accuracy is 0.18219372606394424
calculating predict probability and saving with index to a csv file
saving the model to disk...
--- 224.29870319366455 seconds ---
Start Time = 23:54:51
fitting logistic_regression Model...
calculating accuracy on test...
model accuracy is 0.18006368604501055
calculating predict probability and saving with index to a csv file
saving the model to disk...
--- 226.311585187912 seconds ---
Start Time = 23:58:37
fitting logistic_regression Model...
calculating accuracy on test...
model accuracy is 0.18010671715650414
calculating predict probability and saving with index to a csv file
saving the model to disk...
--- 216.15363597869873 seconds ---
Start Time = 00:02:13
fitting logistic_regression Model...
calculating accuracy on test...
model accuracy is 0.1867550238822669
calculating predict probability and saving with index to a csv file
saving the model to disk..

In [28]:
# Naive_Bayes
model = 'Naive_Bayes'
from sklearn.model_selection import StratifiedKFold, KFold
kf = KFold(n_splits=10, shuffle=True)

i=0

for train_index, test_index in kf.split(X, y):
    i+=1
    import time
    start_time = time.time()   
    from datetime import datetime
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Start Time =", current_time)
    
    # assign training and testing set
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # fit model with a transformation pipeline
    print(f'fitting {model} Model...') 
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import StandardScaler
    from sklearn.naive_bayes import GaussianNB
    pipe = Pipeline([('scaler', StandardScaler()), (f'{model}', GaussianNB() )])
    pipe.fit(X_train, y_train)
    
    # accuracy metrics
    print('calculating accuracy on test...')
    from sklearn.metrics import accuracy_score
    print(f'model accuracy is {accuracy_score(y_test, pipe.predict(X_test))}') 
    
    
    # calculate and save predict probability to csv file
    print('calculating predict probability and saving with index to a csv file')
    y_pred_prob = pipe.predict_proba(X_test)
    y_pred_prob_df = pd.DataFrame(y_pred_prob)
    y_pred_prob_df['index'] = test_index
    y_pred_prob_df.to_csv(f'D:/{model}_pred_prob_test_index_{i}.csv')
    
    # save the model to disk
    print('saving the model to disk...')
    import joblib
    joblib.dump(pipe, f'D:/{model}_labelDataOffset_{i}.sav')
    
    print("--- %s seconds ---" % (time.time() - start_time))
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")

Start Time = 22:03:32
fitting Naive_Bayes Model...
calculating accuracy on test...
model accuracy is 0.15801024140453548
calculating predict probability and saving with index to a csv file
saving the model to disk...
--- 13.343567609786987 seconds ---
Start Time = 22:03:45
fitting Naive_Bayes Model...
calculating accuracy on test...
model accuracy is 0.1580963036275227
calculating predict probability and saving with index to a csv file
saving the model to disk...
--- 13.22922134399414 seconds ---
Start Time = 22:03:58
fitting Naive_Bayes Model...
calculating accuracy on test...
model accuracy is 0.1610869658763286
calculating predict probability and saving with index to a csv file
saving the model to disk...
--- 13.234955310821533 seconds ---
Start Time = 22:04:12
fitting Naive_Bayes Model...
calculating accuracy on test...
model accuracy is 0.1601833125349628
calculating predict probability and saving with index to a csv file
saving the model to disk...
--- 13.311570405960083 seconds 

In [29]:
# Decision_Tree
model = 'Decision_Tree'
from sklearn.model_selection import StratifiedKFold, KFold
kf = KFold(n_splits=10, shuffle=True)

i=0

for train_index, test_index in kf.split(X, y):
    i+=1
    import time
    start_time = time.time()
    from datetime import datetime
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Start Time =", current_time)
    
    # assign training and testing set
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # fit model with a transformation pipeline
    print(f'fitting {model} Model...') 
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import StandardScaler
    from sklearn.tree import DecisionTreeClassifier
    pipe = Pipeline([('scaler', StandardScaler()), (f'{model}', DecisionTreeClassifier() )])
    pipe.fit(X_train, y_train)
    
    # accuracy metrics
    print('calculating accuracy on test...')
    from sklearn.metrics import accuracy_score
    print(f'model accuracy is {accuracy_score(y_test, pipe.predict(X_test))}') 
    
    
    # calculate and save predict probability to csv file
    print('calculating predict probability and saving with index to a csv file')
    y_pred_prob = pipe.predict_proba(X_test)
    y_pred_prob_df = pd.DataFrame(y_pred_prob)
    y_pred_prob_df['index'] = test_index
    y_pred_prob_df.to_csv(f'D:/{model}_pred_prob_test_index_{i}.csv')
    
    # save the model to disk
    print('saving the model to disk...')
    import joblib
    joblib.dump(pipe, f'D:/{model}_labelDataOffset_{i}.sav')
    
    print("--- %s seconds ---" % (time.time() - start_time))
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")

Start Time = 22:05:44
fitting Decision_Tree Model...
calculating accuracy on test...
model accuracy is 0.45849649296441325
calculating predict probability and saving with index to a csv file
saving the model to disk...
--- 660.2064199447632 seconds ---
Start Time = 22:16:44
fitting Decision_Tree Model...
calculating accuracy on test...
model accuracy is 0.4581737596282112
calculating predict probability and saving with index to a csv file
saving the model to disk...
--- 654.8315889835358 seconds ---
Start Time = 22:27:39
fitting Decision_Tree Model...
calculating accuracy on test...
model accuracy is 0.4587761951891217
calculating predict probability and saving with index to a csv file
saving the model to disk...
--- 608.6133470535278 seconds ---
Start Time = 22:37:47
fitting Decision_Tree Model...
calculating accuracy on test...
model accuracy is 0.4608201729850682
calculating predict probability and saving with index to a csv file
saving the model to disk...
--- 643.6609032154083 sec

In [31]:
# MLP
model = 'MLP'
from sklearn.model_selection import StratifiedKFold, KFold
kf = KFold(n_splits=10, shuffle=True)

i=0

for train_index, test_index in kf.split(X, y):
    i+=1
    import time
    start_time = time.time()
    from datetime import datetime
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Start Time =", current_time)
    
    # assign training and testing set
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # fit model with a transformation pipeline
    print(f'fitting {model} Model...') 
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import StandardScaler
    from sklearn.neural_network import MLPClassifier
    pipe = Pipeline([('scaler', StandardScaler()), (f'{model}', MLPClassifier(hidden_layer_sizes=(512,256,128,64,32,16,10),
                                                                              batch_size=128,
                                                                              verbose=10,
                                                                              early_stopping=True,
                                                                              max_iter=1000
                                                                             ) )])
    pipe.fit(X_train, y_train)
    
    # accuracy metrics
    print('calculating accuracy on test...')
    from sklearn.metrics import accuracy_score
    print(f'model accuracy is {accuracy_score(y_test, pipe.predict(X_test))}') 
    
    
    # calculate and save predict probability to csv file
    print('calculating predict probability and saving with index to a csv file')
    y_pred_prob = pipe.predict_proba(X_test)
    y_pred_prob_df = pd.DataFrame(y_pred_prob)
    y_pred_prob_df['index'] = test_index
    y_pred_prob_df.to_csv(f'D:/{model}_pred_prob_test_index_{i}.csv')
    
    # save the model to disk
    print('saving the model to disk...')
    import joblib
    joblib.dump(pipe, f'D:/{model}_labelDataOffset_{i}.sav')
    
    print("--- %s seconds ---" % (time.time() - start_time))
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")

Start Time = 12:21:55
fitting MLP Model...
Iteration 1, loss = 1.43110156
Validation score: 0.457555
Iteration 2, loss = 1.15697157
Validation score: 0.543329
Iteration 3, loss = 1.06120746
Validation score: 0.569434
Iteration 4, loss = 1.00404543
Validation score: 0.592766
Iteration 5, loss = 0.96738862
Validation score: 0.609715
Iteration 6, loss = 0.93512198
Validation score: 0.597021
Iteration 7, loss = 0.90986023
Validation score: 0.627358
Iteration 8, loss = 0.88927266
Validation score: 0.629246
Iteration 9, loss = 0.87181566
Validation score: 0.647725
Iteration 10, loss = 0.85790947
Validation score: 0.646936
Iteration 11, loss = 0.84529793
Validation score: 0.653176
Iteration 12, loss = 0.83357825
Validation score: 0.661447
Iteration 13, loss = 0.82250117
Validation score: 0.650212
Iteration 14, loss = 0.81269027
Validation score: 0.661638
Iteration 15, loss = 0.80520352
Validation score: 0.657383
Iteration 16, loss = 0.79756910
Validation score: 0.665129
Iteration 17, loss = 0

Iteration 49, loss = 0.69169321
Validation score: 0.700748
Iteration 50, loss = 0.69106290
Validation score: 0.706796
Iteration 51, loss = 0.68819028
Validation score: 0.712247
Iteration 52, loss = 0.68871264
Validation score: 0.699720
Iteration 53, loss = 0.68465526
Validation score: 0.711841
Iteration 54, loss = 0.68257538
Validation score: 0.713729
Iteration 55, loss = 0.68084508
Validation score: 0.710956
Iteration 56, loss = 0.68131588
Validation score: 0.711673
Iteration 57, loss = 0.67887191
Validation score: 0.713968
Iteration 58, loss = 0.67770528
Validation score: 0.713538
Iteration 59, loss = 0.67497121
Validation score: 0.705577
Iteration 60, loss = 0.67442346
Validation score: 0.714040
Iteration 61, loss = 0.67164942
Validation score: 0.707633
Iteration 62, loss = 0.67121119
Validation score: 0.712510
Iteration 63, loss = 0.67047981
Validation score: 0.716215
Iteration 64, loss = 0.66733129
Validation score: 0.715546
Iteration 65, loss = 0.66755530
Validation score: 0.7176

Iteration 33, loss = 0.72642191
Validation score: 0.695704
Iteration 34, loss = 0.72275066
Validation score: 0.698119
Iteration 35, loss = 0.72016021
Validation score: 0.696971
Iteration 36, loss = 0.71882974
Validation score: 0.697808
Iteration 37, loss = 0.71441883
Validation score: 0.701298
Iteration 38, loss = 0.71282103
Validation score: 0.691329
Iteration 39, loss = 0.70888293
Validation score: 0.700342
Iteration 40, loss = 0.70610491
Validation score: 0.704047
Iteration 41, loss = 0.70458042
Validation score: 0.702278
Iteration 42, loss = 0.70148599
Validation score: 0.705027
Iteration 43, loss = 0.69940602
Validation score: 0.706940
Iteration 44, loss = 0.69612512
Validation score: 0.701872
Iteration 45, loss = 0.69594006
Validation score: 0.705792
Iteration 46, loss = 0.69304505
Validation score: 0.698382
Iteration 47, loss = 0.69145124
Validation score: 0.699027
Iteration 48, loss = 0.68918959
Validation score: 0.711099
Iteration 49, loss = 0.68757436
Validation score: 0.7100

Iteration 1, loss = 1.44579725
Validation score: 0.437738
Iteration 2, loss = 1.16594821
Validation score: 0.547297
Iteration 3, loss = 1.05155413
Validation score: 0.580383
Iteration 4, loss = 1.00184008
Validation score: 0.566279
Iteration 5, loss = 0.96327706
Validation score: 0.593459
Iteration 6, loss = 0.93570206
Validation score: 0.606560
Iteration 7, loss = 0.91321748
Validation score: 0.614186
Iteration 8, loss = 0.89439382
Validation score: 0.631015
Iteration 9, loss = 0.87750674
Validation score: 0.636275
Iteration 10, loss = 0.86529143
Validation score: 0.642108
Iteration 11, loss = 0.85131242
Validation score: 0.642203
Iteration 12, loss = 0.83954498
Validation score: 0.648897
Iteration 13, loss = 0.82904097
Validation score: 0.655160
Iteration 14, loss = 0.82256315
Validation score: 0.664770
Iteration 15, loss = 0.81438903
Validation score: 0.663575
Iteration 16, loss = 0.80601259
Validation score: 0.663001
Iteration 17, loss = 0.79861970
Validation score: 0.662045
Iterat

Validation score: 0.700677
Iteration 36, loss = 0.70597468
Validation score: 0.704334
Iteration 37, loss = 0.70266776
Validation score: 0.697138
Iteration 38, loss = 0.70009747
Validation score: 0.703784
Iteration 39, loss = 0.69845984
Validation score: 0.709067
Iteration 40, loss = 0.69482173
Validation score: 0.704382
Iteration 41, loss = 0.69375837
Validation score: 0.704621
Iteration 42, loss = 0.69135844
Validation score: 0.698477
Iteration 43, loss = 0.68843237
Validation score: 0.715092
Iteration 44, loss = 0.68571697
Validation score: 0.708518
Iteration 45, loss = 0.68493187
Validation score: 0.708589
Iteration 46, loss = 0.68407156
Validation score: 0.706868
Iteration 47, loss = 0.67971820
Validation score: 0.709211
Iteration 48, loss = 0.67814926
Validation score: 0.704621
Iteration 49, loss = 0.67781605
Validation score: 0.713849
Iteration 50, loss = 0.67427598
Validation score: 0.713801
Iteration 51, loss = 0.67352068
Validation score: 0.709737
Iteration 52, loss = 0.672183

Validation score: 0.702015
Iteration 34, loss = 0.72252937
Validation score: 0.688580
Iteration 35, loss = 0.71969041
Validation score: 0.710621
Iteration 36, loss = 0.71556324
Validation score: 0.700175
Iteration 37, loss = 0.71257952
Validation score: 0.698908
Iteration 38, loss = 0.70958292
Validation score: 0.703760
Iteration 39, loss = 0.70831156
Validation score: 0.709904
Iteration 40, loss = 0.70477312
Validation score: 0.705768
Iteration 41, loss = 0.70235710
Validation score: 0.699123
Iteration 42, loss = 0.70091688
Validation score: 0.704621
Iteration 43, loss = 0.69875340
Validation score: 0.700198
Iteration 44, loss = 0.69692811
Validation score: 0.713131
Iteration 45, loss = 0.69372076
Validation score: 0.712582
Iteration 46, loss = 0.69152652
Validation score: 0.710143
Iteration 47, loss = 0.69004314
Validation score: 0.704262
Iteration 48, loss = 0.68725024
Validation score: 0.708494
Iteration 49, loss = 0.68718818
Validation score: 0.707920
Iteration 50, loss = 0.683979

Iteration 51, loss = 0.68957510
Validation score: 0.712295
Iteration 52, loss = 0.68787684
Validation score: 0.713849
Iteration 53, loss = 0.68710219
Validation score: 0.706055
Iteration 54, loss = 0.68734421
Validation score: 0.712366
Iteration 55, loss = 0.68324325
Validation score: 0.715785
Iteration 56, loss = 0.68245518
Validation score: 0.713610
Iteration 57, loss = 0.68117011
Validation score: 0.714542
Iteration 58, loss = 0.68041290
Validation score: 0.711243
Iteration 59, loss = 0.67854784
Validation score: 0.711052
Iteration 60, loss = 0.67729626
Validation score: 0.716574
Iteration 61, loss = 0.67584925
Validation score: 0.719825
Iteration 62, loss = 0.67563905
Validation score: 0.716837
Iteration 63, loss = 0.67228047
Validation score: 0.715163
Iteration 64, loss = 0.67251507
Validation score: 0.717984
Iteration 65, loss = 0.67112397
Validation score: 0.713992
Iteration 66, loss = 0.66926606
Validation score: 0.712605
Iteration 67, loss = 0.66792304
Validation score: 0.7183